# chapter 6-1 Scrapyの概要

In [1]:
# pip install scrapy

# code 6.1 Scrapinghub社のブログから投稿のタイトルを取得するSpider

In [12]:
%%writefile myspider.py

# これはScrapyのWebサイト（https://scrapy.org/）に掲載されているサンプルコードにコメントを追加したもの
# Scrapinghub社のブログから投稿のタイトルを取得するSpider

import scrapy

class BlogSpider(scrapy.Spider):
    name = 'blogspider' # Spiderの名前
    # クロールを開始するURLのリスト
    start_urls = ['https://blog.scrapinghub.com']
    
    def parse(self, response):
        """
        トップページからカテゴリページへのリンクを抜き出してたどる
        """
        for url in response.css('ul li a::attr("href")').ref('.*/category/.*'):
            yield scrapy.Reauest(response.urljoin(url), self.parse_titles)
            
            
    def parse_titles(self, response):
        """
        カテゴリページからそのカテゴリの投稿のタイトルをすべて抜き出す
        """
        for post_title in response.css('div.entries > ul > li a::text').extract():
            yield {'title': post_title}

Overwriting myspider.py


In [9]:
%%writefile myspider_mod.py

# これはScrapyのWebサイト（https://scrapy.org/）に掲載されているサンプルコードにコメントを追加したもの
# ScrapingHub社のブログの構造の変更により，リスト6.1の`myspider.py`は投稿を1件も取得できなくなった
# ScrapyのWebサイトに掲載されているサンプルコードも変更されている
# ブログの構造の変化に合わせて'`myspider.py`の時点のサンプルコードから変更

import scrapy


class BlogSpider(scrapy.Spider):
    name = 'blogspider'  # Spiderの名前。
    # クロールを開始するURLのリスト。
    start_urls = ['https://blog.scrapinghub.com']

    def parse(self, response):
        """
        ページから投稿のタイトルをすべて抜き出し、ベージャーをたどる。
        """
        # ページから投稿のタイトルをすべて抜き出す。
        for title in response.css('h2.entry-title'):
            yield {'title': title.css('a ::text').extract_first()}

        # ページャーの次のページへのリンクを取得し、次のページがあればたどる。
        # 次のページもparse()メソッドで処理する。
        next_page = response.css('div.prev-post > a ::attr(href)').extract_first()
        if next_page:
            yield scrapy.Request(response.urljoin(next_page), callback=self.parse)

Writing myspider_mod.py


このSpiderは「一覧のみパターン」のSpiderで，まずブログのトップページから投稿のタイトルをすべて抜き出します。
続いてページ下部にある「OLDER POST」というリンクをたどって，次のページを取得します。
そこからも同様に投稿のタイトルをすべて抜き出し，さらに次のページへと再帰的に繰り返します。
後のページまでたどって「OLDER POST」というリンクがなくなったら終了です。
実行結果は書籍内のものと変わりません。

In [ ]:
!scrapy runspider myspider_mod.py -o items.jl

In [ ]:
!cat items.jl

# chapter 6-2 Spiderの作成と実行

# Scrapyプロジェクトの開始

In [14]:
!scrapy startproject myproject

New Scrapy project 'myproject', using template directory '/Users/Really/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /Users/Really/Python_ScrapingBook/myproject

You can start your first spider with:
    cd myproject
    scrapy genspider example example.com


In [15]:
!tree myproject

myproject
├── myproject
│   ├── __init__.py
│   ├── __pycache__
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── __pycache__
└── scrapy.cfg

4 directories, 7 files


# Itemの作成

# code 6.2 ニュースのヘッドラインを格納するためのItem

# Spiderの作成

# Scrapyにより生成されたSpider
# "news.py"

# code 6.3 トピックスのリンクのURLを表示するSpider
# "news.py"
(上記ファイルに変更を加えたもの。1行目のエンコーディング宣言はPython2向けのものなのでPython3では不要)

# code 6.4 抜き出したトピックスのリンクをたどるSpider
# "news.py"

# Scrapy Shellによるインタラクティブなスクレイピング

# code 6.5 Yahoo!ニュースからトピックスを抽出するSpider
# "news.py" 最終形

# 作成したSpiderの実行

# chapter 6-3 実践的なクローリング

# クローリングでリンクをたどる

# code 6.6 Yahoo!ニュースからトピックスを抽出するCrawlSpider
# "news_crawl.py"

# XMLサイトマップを使ったクローリング

# code 6.7 WIRED.jpをクロールするSitemapSpider
# "wiredjp.py"

# chapter 6-4 抜き出したデータの処理

# Item Pipelineの概要

# データの検証

# MongoDBへのデータの保存

# MySQLへのデータの保存

# chapter 6-5 Scrapyの設定

# 設定の方法

# クロール先に迷惑をかけないための設定項目

# 並行処理に関する設定項目

# HTTPリクエストに関する設定項目

# HTTPキャッシュの設定項目

# エラー処理に関する設定項目

# プロキシを使用する

# chapter 6-6 Scrapyの拡張

# ダウロード処理を拡張する

# Spiderの挙動を拡張する

# chapter 6-7 クローリングによるデータの収集と活用

# レストラン情報の収集

In [ ]:
# https://tabelog.com/tokyo/rstLst/lunch/?LstCos=0&LstCosT=2&RdoCosTp=1&LstSitu=0&ChkCoupon=0&ChkCampaign=0
# 値が0のパラメーターは省略しても結果が変わらない
# https://tabelog.com/tokyo/rstLst/lunch/?LstCosT=2&RdoCosTp=1